# Use Raw CSV's created in R to update SDE.Stream with CSCI scores

## Setup

In [ ]:
import arcpy
from datetime import datetime
import os
import sys
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import sqlalchemy as sa
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer
import pandas as pd
import numpy as np
import requests

gis = GIS()
# # Set Pandas display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# in memory output file path
wk_memory = "memory" + "\\"

# set workspace and sde connections 
working_folder = r"F:/Research and Analysis/Fisheries/Streams/Bioassessment/California Stream Condition Index/California Stream Condition Index"
workspace      = r"F:/Research and Analysis/Workspace/Evelyn/Scratch.gdb"
arcpy.env.workspace = r"F:/Research and Analysis/Workspace/Evelyn/Scratch.gdb"

# network path to connection files
filePath = r"F:\GIS\DB_CONNECT"

# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")

# local variables
fdata = os.path.join(sdeBase, "sde.SDE.Monitoring")
## Final feature class to append to in Enterprise Geodatabase
sdeStreams = os.path.join(sdeBase, "sde.SDE.Monitoring\sde.SDE.Stream")
## orginal CSVs that come from preprocessing or R tools created by State
#originalcsv22 = os.path.join(working_folder,"2022_CSCI","19-20NV-22allcore.csv")
#locationcsv22 = os.path.join(working_folder, "2022_CSCI","Stations19_22.csv")
#originalcsv20 = os.path.join(working_folder,"2020_CSCI","core.csv")
#locationcsv20 = os.path.join(working_folder,"2020_CSCI","Stations_20.csv")
#originalcsv23 = os.path.join(working_folder,"2023_CSCI","CSCI_Report_2023.csv")
#locationcsv23 = os.path.join(working_folder,"2023_CSCI","stations_23.csv")

originalcsv24 = os.path.join(working_folder,"2024_CSCI","CSCI_Report_2024.csv")
locationcsv24 = os.path.join(working_folder,"2024_CSCI","Stations_2024.csv")

working_folder = os.path.join(working_folder,"2024_CSCI")

originalcsv = originalcsv24
locationcsv = locationcsv24

if not os.path.exists(originalcsv):
    print(f"Error: File not found at {originalcsv}")

In [ ]:
#Do I need this? -->
#df = get_fs_data('https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8')

In [ ]:
# Function to assign Station type and lat long and LTinfo website to Trend Sites


#Calculate Rating for CSCI value
#Define a function to categorize values based on ranges
def categorize_value(value):
    if 0 <= value < 0.6:
        return 'poor'
    elif 0.6 <= value < 0.8:
        return 'marginal'
    elif 0.8 <= value < 1.0:
        return 'good'
    else:
        return 'excellent'
    
def get_fs_data(service_url):
    
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features

    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])

    return all_data

In [ ]:
#Create Dictionary Using Rest Service data
# setup
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureLayer

# Connect to TRPA Enterprise GIS Portal *if it's a service only shared with org
# portal_user = "TRPA_PORTAL_ADMIN"
# portal_pwd = str(os.environ.get('Password'))
# portal_url = "https://maps.trpa.org/portal/"

# setup connection to GIS server this can be GIS() with a public service
gis = GIS()


# get Stream data as a Spatially Enabled Dataframe
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'
feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()

# Convert the query result to a Spatially Enabled Dataframe
sdfStreamHab = query_result.sdf

sdfStreamHab.info()
columnstokeep = ['SITE_NAME','STATION_TYPE', 'LATITUDE', 'LONGITUDE', 'LTINFO']
sdfStreamHab = sdfStreamHab.loc[:, columnstokeep]
unique_values = sdfStreamHab.drop_duplicates()

# Select specific columns for look up
selected_columns = ['STATION_TYPE', 'LATITUDE', 'LONGITUDE', 'LTINFO']

# Convert selected columns to dictionary
lookup_dict = unique_values.set_index('SITE_NAME')[selected_columns].to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

## Transform CSCI Scores to Point feature class in Enterprise Geodatabase

In [ ]:
# Create DataFrames from CSV files
#For 2024 data add locations of new data and label in similar way
dfCSCI = pd.read_csv(originalcsv)
dflocations = pd.read_csv(locationcsv)

# # Append DataFrames from additional CSV files
# dfCSCI = dfCSCI.append(pd.read_csv(originalcsv), ignore_index=True)
# dflocations = dflocations.append(pd.read_csv(locationcsv), ignore_index=True)

# merge CSCI scores and location data
RawData_df = pd.merge(dfCSCI, dflocations, how='inner', on='StationCode')

try:
    dfCSCI = pd.read_csv(originalcsv)
except FileNotFoundError:
    print("File not found. Please check the file path.")
except pd.errors.EmptyDataError:
    print("The CSV file is empty.")
except pd.errors.ParserError:
    print("Error parsing the CSV file.")

In [ ]:
# get year from sample id
RawData_df['Year']=RawData_df.SampleID.str.split("_").str[-1]

#Calculate Station Type 
RawData_df['STATION_TYPE'] = RawData_df['StationCode'].map(lambda x: lookup_dict[x]['STATION_TYPE'] if x in lookup_dict else 'Status')

#Calculate LATITUDE
RawData_df['LATITUDE'] = RawData_df['StationCode'].map(lambda x: lookup_dict[x]['LATITUDE']if x in lookup_dict else None)
RawData_df['LATITUDE'] = RawData_df['LATITUDE'].fillna(RawData_df['New_Lat'])
#Calculate LONGITUDE
RawData_df['LONGITUDE'] = RawData_df['StationCode'].map(lambda x: lookup_dict[x]['LONGITUDE']if x in lookup_dict else None)
RawData_df['LONGITUDE'] = RawData_df['LONGITUDE'].fillna(RawData_df['New_Long'])
#Caculate LTINFO
RawData_df['LTINFO'] = RawData_df['StationCode'].map(lambda x: lookup_dict[x]['LTINFO'] if x in lookup_dict else None)

#Use only for threshold?
# Apply the categorization function to create the new field
#RawData_df['Rating'] = RawData_df['CSCI'].apply(categorize_value)


Field_Mapping={
    'StationCode': 'SITE_NAME',
    'Year': 'YEAR_OF_COUNT',
    'LATITUDE': 'LATITUDE',
    'LONGITUDE': 'LONGITUDE',
    'CSCI': 'COUNT_VALUE',
    'STATION_TYPE': 'STATION_TYPE',
    'LTINFO': 'LTINFO'
    
}
# rename fields based on field mappings
df_final = RawData_df.rename(columns=Field_Mapping).drop(columns=[col for col in RawData_df.columns if col not in Field_Mapping])

# establish duration field
def assign_duration(stationtype):
    if stationtype == 'Status' :
        return 'One-time'
    else:
        return 'Long-term'
df_final['DURATION']= df_final['STATION_TYPE'].apply(assign_duration)

# station code is site name, site name is station code.
df_final['STATION_CODE']=df_final['SITE_NAME']

# export to csv for QA
df_final.to_csv(os.path.join(working_folder,"StreamCSCI_proccesed.csv"), index= False)

In [ ]:
# convert CSV to point feature class
arcpy.management.XYTableToPoint(os.path.join(working_folder,"StreamCSCI_proccesed.csv"), 
                                "NewCSCI_points", 
                                "LONGITUDE", "LATITUDE",
                                coordinate_system= arcpy.SpatialReference(4269)) #NAD 1983

# project to UTM Zone 10N
arcpy.Project_management("NewCSCI_points", "NewStream_CSCI_Projected", 26910)


# THIS IS HITTING SDE - BE CAREFUL

In [ ]:

inputfc= "NewStream_CSCI_Projected"

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeBase, "ALL")
 
# unregister the sde feature class as versioned
print ("\nUnregistering feature dataset as versioned...")
arcpy.UnregisterAsVersioned_management(fdata,"NO_KEEP_EDIT","COMPRESS_DEFAULT")
print ("\nFinished unregistering feature dataset as versioned.")

arcpy.management.Append(inputfc, sdeStreams,"NO_TEST")

# disconnect all users
print("\nDisconnecting all users...")
arcpy.DisconnectUser(sdeBase, "ALL")

# register SDE feature class as versioned
arcpy.RegisterAsVersioned_management(fdata, "NO_EDITS_TO_BASE")

## Merge data from SDE.Index and SDE.Monitoring to get one place for all stream data

In [ ]:
#This next bit of code was used to add status sites to SDE.Stream so we can consolidate all stream data into one place. This code has been used and SDE.Stream is now the master data set for all stream data.
#I would also like to incorporate IPI scores into SDE.Stream so we have a single place for all stream data. IPI scores are used to calcluate Stream Habitat Condition. 

import os
import pandas as pd
from arcgis.features import FeatureLayer
from arcgis import GIS
import arcpy
from arcgis.features import GeoAccessor
import os

# Set up the GIS connection
gis = GIS()

# set workspace and sde connections 
working_folder = r"F:\Research and Analysis\Fisheries\Streams\Bioassessment\California Stream Condition Index\California Stream Condition Index"
workspace      = r"F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb"
arcpy.env.workspace = r"F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'"

# network path to connection files
filePath = r"F:\GIS\DB_CONNECT"
# network path to connection files
#filePath = r'F:\Research and Analysis\Workspace\Sarah'

# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
# database file path 
#sdeBase    = os.path.join(filePath, "Vector.sde")
#sdeCollect = os.path.join(filePath, "Collection.sde")

arcpy.env.workspace = workspace


# Paths to feature classes
CSCIfromIndex = os.path.join(sdeBase, "SDE.Index", "SDE.StreamHabitat_CSCI")


# Create a DataFrame from SDE.StreamHabitat_CSCI
sdfCSCIfromIndex = pd.DataFrame.spatial.from_featureclass(CSCIfromIndex)

# Define columns for the new DataFrame
columns_for_new_df = ['StationCod', 'F2000_score', 'F2007_score', 'F2008_score', 'F2009_score', 'F2010_score',
                       'F2011_score', 'F2012_score', 'F2013_score', 'F2014_score', 'F2015_score', 'F2016_score',
                       'F2017_score', 'F2018_score', 'F2019_score', 'LATITUDE', 'LONGITUDE']

# Create a new DataFrame using the defined columns
wide_df = pd.DataFrame(sdfCSCIfromIndex, columns=columns_for_new_df)

# Melt the DataFrame to convert from wide to long format
long_df = pd.melt(wide_df, id_vars=['StationCod', 'LATITUDE', 'LONGITUDE'], var_name='Year', value_name='CSCI')

long_df = long_df[(long_df['CSCI'] != 0) & (~long_df['CSCI'].isnull())]

# Extract the year from the 'Year' column
long_df['Year'] = long_df['Year'].str.extract(r'(\d{4})')

#print("Wide DataFrame:")
#print(wide_df)

#print("\nLong DataFrame:")
#print(long_df)

# Use a lookup dictionary to filter out any data that is already present in SDE.Streams
# Create a dictionary using Rest Service data
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'
feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()

# Convert the query result to a Spatially Enabled DataFrame
sdfStreamHab = query_result.sdf

# Select specific columns for lookup
columns_to_keep = ['SITE_NAME', 'STATION_TYPE', 'LATITUDE', 'LONGITUDE', 'LTINFO']
sdfStreamHab = sdfStreamHab.loc[:, columns_to_keep]

# Remove duplicates
unique_values = sdfStreamHab.drop_duplicates()

# Convert selected columns to a dictionary
lookup_dict = unique_values.set_index('SITE_NAME').to_dict(orient='index')

# Filter out rows where 'StationCod' is in the lookup dictionary
filtered_df = long_df[~long_df['StationCod'].isin(lookup_dict)]



# Add new fields and fill them with values
filtered_df = filtered_df.copy()
filtered_df.loc[:, 'Duration'] = 'One-time'
filtered_df.loc[:, 'STATION_TYPE'] = 'Status'
filtered_df.loc[:, 'LTINFO'] = 'Null'
filtered_df['STATION_CODE'] = filtered_df['StationCod']
#filtered_df.loc[:, 'STATION_CODE'] = sdfStreamHab['SITE_NAME']
# Drop rows with null values in the COUNT_VALUE column
filtered_df.dropna(subset=['CSCI'], inplace=True)

print("Filtered DataFrame:")
print(filtered_df)

# Field Mapping
field_mapping = {
    'StationCod': 'SITE_NAME',
    'STATION_CODE': 'STATION_CODE',
    'Year': 'YEAR_OF_COUNT',
    'LATITUDE': 'LATITUDE',
    'LONGITUDE': 'LONGITUDE',
    'CSCI': 'COUNT_VALUE',
    'STATION_TYPE': 'STATION_TYPE',
    'LTINFO': 'LTINFO'
}

# Rename fields based on field mappings
ready_df = filtered_df.rename(columns=field_mapping).drop(columns=[col for col in filtered_df.columns if col not in field_mapping])

# Drop rows with null values in the COUNT_VALUE column
#ready_df.dropna(subset=['COUNT_VALUE'], inplace=True)

# Convert DataFrame to Spatially Enabled DataFrame
3#sedf = GeoAccessor.from_xy(ready_df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'CSCIfromIndex_Stagingg'), sanitize_columns=False)

# Specify the input feature class (CSCIfromIndex_Staging) and the output projected feature class name
#input_feature_class = os.path.join(workspace, 'CSCIfromIndex_Stagingg')
#output_projected_feature_class = os.path.join(workspace, 'CSCIfromIndex_Projected_ready')

# Specify the output spatial reference system (SR) using either a well-known ID (WKID) or a path to a .prj file
#output_spatial_reference = arcpy.SpatialReference(26910)  

# Project the feature class
#arcpy.Project_management(input_feature_class, output_projected_feature_class, output_spatial_reference)

#print("Projection completed.")


# Export to CSV
ready_df.to_csv(os.path.join(working_folder, "OldCSCIdata.csv"), index=False)

csv_path = r"F:\Research and Analysis\Fisheries\Streams\Bioassessment\California Stream Condition Index\California Stream Condition Index\OldCSCIdata.csv" 

# Convert CSV to point feature class
arcpy.management.XYTableToPoint(csv_path, "oldStream_CSCI_Points", "LONGITUDE", "LATITUDE")

# Project to UTM Zone 10N
arcpy.Project_management("oldStream_CSCI_Points", "old_CSCI_Projected", arcpy.SpatialReference(26910))

#print("Conversion and projection completed.")


In [ ]:
print(filtered_df['StationCod'].unique())


In [ ]:
#convert CSV to feature class
# Convert DataFrame to CSV
#csv_path = os.path.join(working_folder, "OldCSCIdata.csv")
#ready_df.to_csv(csv_path, index=False) 

# Convert CSV to point feature class
#arcpy.management.XYTableToPoint(csv_path, "NewStream_CSCI_Points", "LONGITUDE", "LATITUDE")

# Project to UTM Zone 10N
#arcpy.Project_management("NewStream_CSCI_Points", "NewStream_CSCI_Projected", arcpy.SpatialReference(26910))

#print("Conversion and projection completed.")

In [ ]:
from arcgis import GIS

# Set up the GIS connection
gis = GIS()

# File paths and workspace
workspace = r"C:\GIS\Scratch.gdb"
arcpy.env.workspace = workspace

# CSV file path
csv_path = r"F:\Research and Analysis\Fisheries\Streams\Bioassessment\California Stream Condition Index\California Stream Condition Index\OldCSCIdata.csv"

# Read CSV into a Pandas DataFrame
df = pd.read_csv(csv_path)

# Convert the Pandas DataFrame to a Spatially Enabled DataFrame
sdf = pd.DataFrame.spatial.from_df(df)

# SDEindexdata feature class name (change it as needed)
SDEindexdata_fc = "OldCSCIdata"

# Save the Spatially Enabled DataFrame to a feature class in the file geodatabase
sdf.spatial.to_featureclass(location=os.path.join(workspace, SDEindexdata_fc))

print(f"CSV data imported to feature class '{SDEindexcscidata_fc}' in '{workspace}'.")